#Introduction of the code

In the following section we wish to create a search engine that can be used on pdf's and get the relevant information regarding the theory used in the pdf, the credits of the pdf, the main arguemnt in the pdf, the method used in the pdf and lastly the conclusion

The idea behind the construction of this application, is that it can be used to save significant time when researching information for a paper, litterature review and ect.

## Upload the packages

Installing LangChain is easy. You can install it with pip:

In this section we install all the packages that are needed to create a chatbot RAG system chatbot function

In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.33.2 --progress-bar off
!pip install -qqq langchain==0.0.299 --progress-bar off
!pip install -qqq chromadb==0.4.10 --progress-bar off
!pip install -qqq xformers==0.0.21 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq tokenizers==0.14.0 --progress-bar off
!pip install -qqq optimum==1.13.1 --progress-bar off
!pip install -qqq auto-gptq==0.4.2 --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ --progress-bar off
!pip install -qqq unstructured==0.10.16 --progress-bar off

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the foll

In [2]:
!pip install accelerate --q

#Model setup

In the following section we will make ud of the Llama 2 hugging face model. Moreover, we choose to simplify the construction of out model by using the HuggingFacePipeline which is a wrapper that is obtained from LangChain. Lastly we specify that we use a GPTQ version of the code, which means that we are working with a pre-trained transformer.

In [3]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
#Firstly, we dine and import the transformer using the "Model_Name" as a defination
#It is nescassary to specify that Llama 2 is a pre-trained and pre-fine-tuned model.
#It is a generative text model that is trained on 2 trillion tokens and has doble the context length produced by meta
MODEL_NAME = "TheBloke/Llama-2-7B-Chat-GPTQ"


# In the below line of code the tokenizer linked to the Llama 2 is initialized
#When using the "AutoTokenizer" function it is possible to create an approach
#to load the tokenizers tailored to a divers range of pre-trained language models

#from the "from_pretrained" function of the below code makes it possible to load a pre-trained tonenizer
# specifcally for the model that we have choosen to work with

#Lastly, from the "use_fast" function that is set to TRUE which is a way to indicate, that we wish
#to use the fastest possible tokenizer, to save time
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)


#In the below line of code the "AutoModelForCausalLM" fucntion is utilized to model the model
#this specific function is designed so to predict the next work in a sequence
#the from "pre-trained" is utilized to load the pre-trained tokenizer for the specific model
#"MODEL_NAME" ia the before defined transformer
#the "torch_dtype" is used to optimize memory usage. This is done by setting the function to float16
#which indicates that the memory of the model will be relative smaller then when setting it to float32
#this can however result in loss of predicion of the models calculations
#The "trust_remote_code" function has been set to TRUE which gratns the model permission to load
#the model if it is hosted remotely. This can for instancely be on the Hugging Face Model hub
#The "device_map" fucntion is set to "auto" which assignes the device for the model loading
#by doing this, the library is able to select an available devise automatically


model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto"
)



# Create a configuration for text generation based on the specified model name - it initializes the settings for the
# pre-trained language model, soecified earlier in the code "Moedl_NAME"
# the "GenerationConfig" function is a class from Hugging Face's transformer library
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

# Set the maximum number of new tokens in the generated text to 1024.
# This means that the limits the length of the generated output to 1024 tokens.
#in the below code the "max_new_tokens" function indicates the upper bounds of the tokens the model is able to
#generate in one generation step
generation_config.max_new_tokens = 1024


#In the below code the models temparature is set to 0.0001
#this means that the randomness of the model is set to a lower value, which makes the output of the model is said to be more deterministic
#Lower values (e.g., 0.0001) make output more deterministic, following likely predictions.
# Higher values make the output more random.
generation_config.temperature = 0.0001


#In the below code the models top_p paramter is set to 0.95
# A value of 0.95 means focusing on the most likely words that make up 95% of the probability distribution.
generation_config.top_p = 0.95


#In the below code the models sampling is set to TRUE, which makes it possible to introduce sampling in the model
# When set to True, the model randomly selects words based on their probabilities, introducing randomness.
generation_config.do_sample = True


#In the below code the modelsrepetion_penaly is set to 1.15
# A value of 1.15 discourages the model from repeating the same words or phrases too frequently in the output.
#This makes it possible for the output of the model to be more dicerse in its nature
generation_config.repetition_penalty = 1.15

#In the below line of code, we create the pipeline for the language model
#Here it can be seen that the configorations made in the above section is set into the model struction
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
)

# In the below section the LangChain pipeline that wraps the text generation pipeline and set a specific temperature for generation
# in the Hugging face pipeline the temperature is set to 0 which means that the generated output should be more deterministic
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]


# Simple Retrieval Augmented Generation (RAG)

To work with external files, LangChain provides data loaders that can be used to load documents from various sources. Combining LLMs with external data is generally referred to as Retrieval Augmented Generation (RAG).

Let's see how we can use the UnstructuredMarkdownLoader to load a document from a Markdown file:

## import package

Firstly we import a package that is necessary to be able to upload a pdf to the database we are creating in this section of the code

In [4]:
!pip install langchain pypdf --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 5.0 MB/s eta 0:00:00


In [6]:
 from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.document_loaders import PyPDFLoader

#In the below code, the "PyPDFLoader" function makes it possible to handle the loading or procession of a PDF-file
#the "()" is the part where you insert the specific "path" for the pdf when it has been oploaded to the colab
loader = PyPDFLoader("/content/Monetary policy af F S Mishkin.pdf")

#In the below code we "load" the object and the pdf that we defined in the aboce code
#this is then saved in the "docs" function
docs = loader.load()

#In the below code, the lenght of the pdf is measured - this measure indacted the number of the pages loaded from the pdf
len(docs)

5

The file used in the this colab is a PDF called "Monetary policy" made by Frederic S. Mishkin

In the following section we are interested to split the document that has been uploaded in the "loader" function into smaller chuncks using the "RecursiveCharecterTextSplitter"

Firstly we import the RecursiveCharacterTextSplitter

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

#In the below code, the text_splitter is set up
# The "chunk_size" function is set to 1024 and indicates the size of text chunks in charecter
# The "chunck_overlap" is det to 64 which determines the size of the overlaps
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)

#In the below code, we set up the text splitter
texts = text_splitter.split_documents(docs)

#In the below code, the lenght of the text chunks generated is measured - this number should be larger then the "len(docs)" function
# this is due to, that the chuncks are generated based on the uploaded pdf
len(texts)

27

## Creation of embeddings

Splitting the document into chunks is required due to the limited number of tokens a LLM can look at once (4096 for Llama 2). Next, we'll use the HuggingFaceEmbeddings class to create embeddings for the chunks:

Firstly, the HuggingFaceEmbedding embedder is imported

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings


#In the below code, the embedding function is set up - these will later be inserted into the databse
#In the below code it is specified that we make use of the gte-large embedding model, which is a text embedding model found on Hugging Face platform
#This specific model is based on the BERT framework
#Next, the "model_kwargs" function is used to define which core that is used. In this embedding model it is set CUDA
#the "encode_kwargs" function is set so it normalize the embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)


#in the below code, the embidding model is generated
query_result = embeddings.embed_query(texts[0].page_content)
print(len(query_result))

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

1024


In the spirit of using free tools, we're also using free embeddings hosted by HuggingFace. We'll use Chroma database to store/cache the embeddings and make it easy to search them:

To combine the LLM with the database, we'll use the RetrievalQA chain:

In [34]:
from langchain.vectorstores import Chroma


#In the below code, we set up the database where the embeddings is going to be stored
#this is done using the chroma
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

#In the below code, a smiliarity check is performed
#this is done so that we can se if the embeddings created and stored in the database "db" is working as required
# moreover, in the below code, the "k" is set to 2, which is a way to determine the specific number of similarity we want to retrieve from the text
results = db.similarity_search("inflation", k=2)
print(results[0].page_content)

announcement of numerical targetsfor inflation for the year ahead. This isespecially important in emerging mar-ket countries, because many of thesecountries routinely reported numericalinflation targets or objectives as partof the government’s economic planfor the coming year and yet their mon-etary policy strategy should not becharacterized as inflation targeting,which requires the other four elementsfor it to be sustainable over the medi-um term.
10Since 1990, inflation tar-
geting has been adopted by manyindustrialized countries (New Zealand,Canada, the United Kingdom, Sweden,Israel, Australia, and Switzerland), byseveral emerging market countries(Chile, Brazil, Korea, Thailand, andSouth Africa), and by several transi-tion countries (Czech Republic,Poland, and Hungary).
Inflation targeting has several


#Creating the template

We will in this section create the initial prompt template for the pdf search engine

In [10]:
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
#firstly the template for the search engine is created
# the template is used to determine how engine should ansswer based on the subsequently prompt
#giving that we would like to the engine to produce an output that is relative neutral and only give us an answer there
# is non emotional coded, the template is produced rather generic
template = """
<s>[INST] <<SYS>>
Use the following information to answer the question at the end.
<</SYS>>

{context}

{question} [/INST]
"""
#Next, the Prompt template is is created.
#This is done using the "PromptTemplate" function, where we also need to include the before created template
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

#in the below code, the retrival function is created
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)
#we start by making a "pratice" question to the search engine so we can figure out if the engine works
#this is also known as the prompt or the "question" we ask the created engine
result = qa_chain(
    "What is the main arguemtns of the article? Explain this in 2-3 sentence."
)
print(result["result"].strip())

The main argument of the article is that monetary policy should be guided by an explicit inflation target, rather than being based solely on economic conditions or central bank discretion. The author, Frederic S. Mishra, argues that this approach, known as "inflation targeting," allows for more predictability and accountability in monetary policy decision-making, and can help to anchor expectations and reduce the risk of high inflation. Additionally, he suggests that using an inflation target can help to improve the communication and transparency of monetary policy, making it easier for households and businesses to understand the central bank's objectives and actions.


This will pass our prompt to the LLM along with the top 2 results from the database. The LLM will then use the prompt to generate an answer. The answer will be returned along with the source documents.

#Prompt engineering

In the following section a fine tuning of the prompts will be performed. Here the self-consistency approach will be employed due to wanting to ensure the reliability of the output of the prompts.

This specific technique is used by asking the model the same quiestion multiple times or in different ways to ensure consistent and reliable responses. This is done with prompts that would like to focus on the theory, credits, main argument, method and conclusion of the choosen pdf.

##Theory prompt

In the following prompt the main theory is the wanted output of the prompt

In [11]:
from textwrap import fill

result = qa_chain(
    "Which theoretical framework serves as the cornerstone in the article? Elaborate on this in 2-3 sentences."
)
print(fill(result["result"].strip(), width=80))

According to the article, the theoretical framework that serves as the
cornerstone is "inflation targeting." Inflation targeting is a monetary policy
strategy that involves setting a clear and publicly announced inflation target,
using monetary policy instruments to achieve the target, and regularly
monitoring and reporting on progress towards the target. This approach is based
on the idea that a clearly defined inflation target can help anchor expectations
and improve the effectiveness of monetary policy.


In [12]:
result = qa_chain(
    "What is the main theoretical perspective utilized in the article? Exaplin this in 2-3 sentences."
)
print(fill(result["result"].strip(), width=80))

The main theoretical perspective utilized in the article is the New Classical
Macroeconomics (NCM) framework, which emphasizes the importance of rational
expectations and the limitations of monetary policy in achieving its goals. The
author uses insights from NCM to analyze the conduct of monetary policy and to
evaluate different strategies for improving the effectiveness of policy.
Specifically, the author draws on the idea that monetary policy must take into
account the expectations of economic agents in order to achieve its desired
effects.


In [13]:
from textwrap import fill

result = qa_chain(
    "What is the principal theoretical framework applied in the article? Explain this in 2-3 sentences."
)
print(fill(result["result"].strip(), width=80))

The principal theoretical framework applied in the article is the concept of
"constrained discretion," which was coined by Bernanke and his colleagues to
describe the nature of inflation targeting. This framework posits that central
banks have limited flexibility in setting monetary policy due to various
constraints, including the zero lower bound on interest rates, political
considerations, and the need to maintain financial stability. The author uses
this framework to analyze the strategies and tactics employed by central banks
around the world in conducting monetary policy.


In [14]:
from textwrap import fill

result = qa_chain(
    "From a theoretical standpoint, what is the main idea of the article? Explain this in 2-3 sentences."
)
print(fill(result["result"].strip(), width=80))

The main idea of the article is that monetary policy should be guided by an
explicit inflation target, which serves as a benchmark for setting interest
rates and other monetary policies. The author argues that this approach, known
as "inflation targeting," allows central banks to use their discretion more
effectively and avoid the pitfalls of past monetary policy frameworks. By
focusing on a clear and measurable objective, such as price stability, central
banks can better navigate complex economic conditions and maintain confidence in
the currency.


In [15]:
from textwrap import fill

result = qa_chain(
    "What constitutes the primary theoretical framework employed in the article? Provide a concise explanation in 2-3 sentences."
)
print(fill(result["result"].strip(), width=80))

The primary theoretical framework employed in the article is the concept of
"constrained discretion," which was coined by Bernanke and his colleagues to
describe the nature of inflation targeting. This framework posits that central
banks have limited flexibility in setting monetary policy due to various
constraints, including the zero lower bound on interest rates, political
considerations, and the need to maintain confidence in the currency. The author
uses this framework to analyze the conduct of monetary policy in different
countries and to identify potential improvements in the strategy and tactics
used by central banks.


## Credit prompt

In the following prompt the cridits is the wanted output of the prompt

In [16]:
result = qa_chain(
    "What are the credits of the article? Write only the information that is needed for the table of contents."
)
print(fill(result["result"].strip(), width=80))

* Author: Frederic S. Mishkin * Article Title: Monetary Policy * Publication:
NBER Reporter Online * Issue: Winter 2001/02 * Publisher: National Bureau of
Economic Research (NBER), Cambridge, MA


In [17]:
result = qa_chain(
    "List the essential credits of the article, including only the information necessary for the table of contents."
)
print(fill(result["result"].strip(), width=80))

| Source | Year | Page Number | | Bank of Canada | 2001 | 203-22 | | F.S.
Mishkin and M. Savastano | 2001 | NBER Working Paper No. 8397 | | Loayza and R.
Soto (eds.) | Forthcoming | N/A | | B.S. Bernanke and F.S. Mishkin | 2001 | NBER
Reporter Online | | F.S. Mishkin and A.S. Posen | 2001 | NBER Reporter Online |
| National Bureau of Economic Research (NBER) | Cambridge, Mass. | N/A |  Note:
The page numbers are not provided in the original text, so they have been left
blank.


In [18]:
result = qa_chain(
    "Extract only the essential credits of the article necessary for the table of contents."
)
print(fill(result["result"].strip(), width=80))

* Mishkin, F. S. (2001). Monetary policy. NBER Reporter Online, National Bureau
of Economic Research (NBER), Cambridge, MA, Iss. Winter 2001/02, pp. 8-11. *
Provided in cooperation with: National Bureau of Economic Research (NBER),
Cambridge, MA.


In [19]:
result = qa_chain(
    "Provide the necessary credits for the article, containing only the essential details required for the table of contents."
)
print(fill(result["result"].strip(), width=80))

The article "Monetary Policy" by Frederic S. Mishkin was published in the NBER
Reporter Online in winter 2001/02. The article can be found at
<http://hdl.handle.net/10419/61807>. According to the terms of use provided by
EconStor, the document may be saved and copied for personal and scholarly
purposes, but it cannot be used for public or commercial purposes without
permission.


In [20]:
result = qa_chain(
    "Enumerate the necessary article credits, highlighting only the details essential for the table of content."
)
print(fill(result["result"].strip(), width=80))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


* National Bureau of Economic Research (NBER), Cambridge, Mass. * Leibniz
Information Centre for Economics, Germany * zbw - Leibniz-Informationszentrum,
Germany * Mishkin, Frederic S. * Article * Monetary policy * NBER Reporter
Online * Provided in cooperation with: * Standard usage conditions: * The
documents on EconStor can be saved and used for scientific purposes and personal
use. * You are not allowed to reproduce or distribute the documents publicly
without permission.


##Main argument prompt

In the following prompt the main argument of the article is the wanted output of the prompt

In [21]:
result = qa_chain(
    "Summarize the main arguments used in the article in 2-3 sentences."
)
print(fill(result["result"].strip(), width=80))

The article discusses the use of inflation targeting as a monetary policy
framework, highlighting its advantages and challenges. The authors argue that
inflation targeting has been successful in many countries, including those with
high inflation rates, by providing a clear and transparent anchor for monetary
policy decisions. However, they also acknowledge that implementing an inflation
targeting regime can be difficult, particularly in emerging market countries
with limited institutional capacity and data availability.


In [22]:
result = qa_chain(
    "Provide a concise summary of the key arguments presented in the article, encapsulating them in 2-3 sentences."
)
print(fill(result["result"].strip(), width=80))

In his article, "Issues in Inflation Targeting," published in the NBER Reporter
Winter 2001/2002, Francis S. Mishkin presents several key arguments related to
inflation targeting as a monetary policy framework. He emphasizes that while
inflation targeting has been successful in some countries, it is not without its
challenges and limitations. Specifically, he highlights issues such as the
difficulty in predicting future inflation rates, the potential for second-round
effects of changes in interest rates on inflation, and the need for careful
communication between central banks and the public regarding their inflation
targets.


In [23]:
result = qa_chain(
    "In 2-3 sentences, distill the core arguments presented in the article into a concise summary."
)
print(fill(result["result"].strip(), width=80))

The article discusses the evolution of monetary policy strategies and tactics,
particularly in the context of inflation targeting. The author, Ben Bernanke,
argues that central banks should adopt a "constrained discretion" approach,
which involves setting clear inflation targets and using various tools to
achieve them while taking into account the limitations of monetary policy.
Bernanke emphasizes the importance of understanding the international experience
with monetary policy strategies and tactics, including the use of unconventional
measures during times of crisis.


In [24]:
result = qa_chain(
    "Condense the article's main arguments into a brief 2-3 sentence summary."
)
print(fill(result["result"].strip(), width=80))

The article argues that inflation targeting has become the dominant monetary
policy framework in industrialized countries, as it provides a clear and
transparent anchor for monetary policy decisions. The authors, including Federal
Reserve Chairman Alan Greenspan, argue that while there are different approaches
to inflation targeting, such as the use of monetary aggregates or direct control
of the money supply, the core idea is to give central banks more flexibility in
setting interest rates while maintaining a stable price level. They also
emphasize the importance of communication and transparency in implementing an
inflation-targeting regime effectively.


##Method prompt

In the following prompt the method of the article is the wanted output of the prompt

In [25]:
result = qa_chain(
    "What is the method used in the article? Explain in 2-3 sentences."
)
print(fill(result["result"].strip(), width=80))

The method used in the article appears to be a combination of summary and
analysis of existing research on inflation targeting. The authors provide a
brief overview of the concept of inflation targeting and its increasing
popularity among central banks around the world. They then analyze various
studies and papers related to inflation targeting, including those by leading
economists such as Ben Bernanke and Francisco Mishkin. Finally, they draw
conclusions based on their analysis and highlight some key lessons learned from
the international experience with inflation targeting.


In [26]:
result = qa_chain(
    "Specify the methodology utilized in the article and elaborate on it in 2-3 sentences."
)
print(fill(result["result"].strip(), width=80))

The methodology used in the article is primarily based on a review and analysis
of existing literature on monetary policy strategies and their implementation in
various countries around the world. The authors conducted a comprehensive search
of academic journals, books, and other publications to identify relevant studies
and papers related to this topic. They then synthesized the findings from these
sources and presented them in a structured manner, highlighting key trends,
challenges, and lessons learned from the international experience.


In [27]:
result = qa_chain(
    "Describe the article's chosen methodology, write it in 2-3 sentences."
)
print(fill(result["result"].strip(), width=80))

The article presents a comprehensive review of the literature on monetary policy
strategies and tactics, drawing on experiences from both developed and emerging
economies. The author employs a systematic approach to analyze the various
methods used by central banks to control inflation, stabilize output, and
promote financial stability. The methodology involves examining the different
approaches to monetary policy, including monetary targeting, inflation
targeting, and others, and assessing their effectiveness through empirical
analysis and case studies.


##Conclusion prompt

In the following prompt the conclusion of the article is the wanted output of the prompt

In [28]:
result = qa_chain(
    "What is the conclusion of the article? Explain in 2-3 sentences."
)
print(fill(result["result"].strip(), width=80))

According to the article, the author concludes that monetary policy has played a
crucial role in stabilizing the economy during times of crisis, such as
recessions or inflation episodes. The author highlights the importance of
monetary policy in maintaining price stability and economic growth, and notes
that central banks have employed various strategies to achieve these goals.
However, the author also acknowledges the limitations of monetary policy and the
need for other policies, such as fiscal policy, to complement its effects.


In [29]:
result = qa_chain(
    "What conclusion does the article reach? Offer a concise 2-3 sentence explanation."
)
print(fill(result["result"].strip(), width=80))

The article concludes that monetary policy has been successful in achieving its
objectives, particularly price stability, but there are still challenges ahead
due to the complexities of modern economies. The author highlights the
importance of inflation targeting as a framework for monetary policy, which
allows central banks to set clear goals and communicate them effectively to the
public.


In [30]:
result = qa_chain(
    "What are the key conclusions drawn in the article? Present a 2-3 sentence explanation"
)
print(fill(result["result"].strip(), width=80))

According to the article, the author's research over the past few years has
focused on monetary policy strategies and tactics used by central banks around
the world, including the United States, emerging markets, and other
industrialized countries. The author concludes that there is no one-size-fits-
all approach to monetary policy, and that each country must tailor its strategy
based on its unique economic conditions and circumstances. Additionally, the
author highlights the importance of communication and transparency in monetary
policy making, as well as the need for central banks to continually adapt their
policies in response to changing economic conditions.


In [31]:
result = qa_chain(
    "Specify the article's conclusion, providing a concise 2-3 sentence explanation"
)
print(fill(result["result"].strip(), width=80))

The article concludes by highlighting the importance of monetary policy in
controlling inflation and maintaining economic stability. The author emphasizes
that monetary policy must be flexible and adaptable to changing economic
conditions, and that central banks must communicate their inflation targets
clearly and consistently to ensure public understanding and support. Finally,
the author notes that while inflation targeting has been successful in many
countries, it is not without challenges and risks, and that central banks must
carefully manage their monetary policies to achieve their inflation objectives
while also considering other important economic goals.


In [32]:
result = qa_chain(
    "Share the article's conclusion in a brief 2-3 sentence explanation."
)
print(fill(result["result"].strip(), width=80))

According to Frederic S. Mishkin, monetary policy plays a crucial role in
controlling inflation and maintaining economic stability. He argues that central
banks should focus on inflation targeting, which involves setting an explicit
inflation rate goal and using monetary policy tools to achieve it. By adopting
this approach, central banks can better manage inflation expectations and ensure
price stability, thereby contributing to overall macroeconomic stability.
